<a href="https://colab.research.google.com/github/AfsarAI/ML-Assignment-1/blob/main/IITM_MLP_KA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

mlp_term_2_2025_kaggle_assignment_1_path = kagglehub.competition_download('mlp-term-2-2025-kaggle-assignment-1')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Datasets

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test_df = pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-1/test.csv')
train_df = pd.read_csv('/kaggle/input/mlp-term-2-2025-kaggle-assignment-1/train.csv')

In [ ]:
train_df.head()

In [ ]:
test_df.head()

# Identifing data types of different columns

In [ ]:
train_df.shape

In [ ]:
train_df.info()

In [ ]:
test_df.info()

# Descriptive statistics of numerical columns

In [ ]:
train_df.loc[:, ['duration', 'days_left']].describe().T

In [ ]:
test_df.loc[:, ['duration', 'days_left']].describe().T

# Identifying and handling the missing values

In [ ]:
train_df.isna().sum()

In [ ]:
test_df.isnull().sum()

In [ ]:
cat = ['airline', 'departure', 'stops']
num = ['duration', 'days_left']

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [ ]:
ct = ColumnTransformer(
    transformers=[
        ('cat', SimpleImputer(strategy='most_frequent'), cat),
        ('num', SimpleImputer(strategy='mean'), num)
    ],
    remainder='passthrough',
    verbose_feature_names_out=False
).set_output(transform='pandas')

In [ ]:
pipe = Pipeline([
    ('ct', ct)
])

In [ ]:
X_train = train_df.drop(columns=['price'])
y_train = train_df['price']

In [ ]:
train_df_t = pipe.fit_transform(X_train)

In [ ]:
test_df_t = pipe.transform(test_df)

## Just for showing all imutations

In [ ]:
original_order = X_train.columns
train_df_t.reindex(columns=original_order)

In [ ]:
original_order = test_df.columns
test_df_t.reindex(columns=original_order)

# Identifying Duplicates

In [ ]:
# Check total duplicates in train dataset
duplicate_rows = train_df_t[train_df_t.duplicated()]
print("Duplicate Rows:", duplicate_rows.shape[0])

In [ ]:
# Check total duplicates in test dataset
duplicate_rows = test_df_t[test_df_t.duplicated()]
print("Duplicate Rows:", duplicate_rows.shape[0])

# Identifying and handling outliers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

### Train Dataset

In [ ]:
numerical_cols = train_df_t.select_dtypes(include=['int64', 'float64'])

for col in numerical_cols:
    plt.figure(figsize=(6, 1))
    sns.boxplot(x=train_df_t[col])
    plt.title(f'Outliers in {col}')
    plt.show()

In [ ]:
Q1 = train_df_t['duration'].quantile(0.25)
Q3 = train_df_t['duration'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = train_df_t[(train_df_t['duration'] < lower) | (train_df_t['duration'] > upper)]
print("Outliers:", outliers.shape[0])
print("Total:", train_df_t.shape[0])
print(f"{(outliers.shape[0]/train_df_t.shape[0])*100} %")

In [ ]:
def cap_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper, upper,
                np.where(df[col] < lower, lower, df[col]))
    print('Capped')
    return df

train_df_t = cap_outliers_iqr(train_df_t, 'duration')

In [ ]:
sns.boxplot(x=train_df_t['duration'])
plt.title("Outliers in Duration")
plt.show()

### Test Dataset

In [ ]:
numerical_cols = test_df_t.select_dtypes(include=['int64', 'float64'])

for col in numerical_cols:
    plt.figure(figsize=(6, 1))
    sns.boxplot(x=test_df_t[col])
    plt.title(f'Outliers in {col}')
    plt.show()

In [ ]:
Q1 = test_df_t['duration'].quantile(0.25)
Q3 = test_df_t['duration'].quantile(0.75)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

outliers = test_df_t[(test_df_t['duration'] < lower) | (test_df_t['duration'] > upper)]
print("Outliers:", outliers.shape[0])
print("Total:", test_df_t.shape[0])
print(f"{(outliers.shape[0]/test_df_t.shape[0])*100} %")

In [ ]:
def cap_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] > upper, upper,
                np.where(df[col] < lower, lower, df[col]))
    print('Capped')
    return df

test_df_t = cap_outliers_iqr(test_df_t, 'duration')

In [ ]:
sns.boxplot(x=test_df_t['duration'])
plt.title("Outliers in Duration")
plt.show()

### Outliers checking in Target

In [ ]:
sns.boxplot(x=y_train)
plt.title("Outliers in Price")
plt.show()

In [ ]:
sns.histplot(y_train, kde=True)
plt.title("Before Log Transform")
plt.show()

print("Skewness:", y_train.skew())

In [ ]:
y_train = np.log1p(y_train)  # log(1 + Price)

In [ ]:
X = train_df_t
y = y_train
print(X, y)

# Visualizations and Graphs with main data set

In [ ]:
avg_price_by_source = train_df.groupby('source')['price'].mean().sort_values()
avg_price_by_source.plot(kind='bar', color='skyblue')
plt.title("Average Price by Source City")
plt.ylabel("Average Ticket Price")
plt.xlabel("Source")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.boxplot(x='airline', y='price', data=train_df)
plt.xticks(rotation=45)
plt.title("Ticket Price Distribution by Airline")
plt.show()

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(x='class', y='price', data=train_df, palette='Set2')
plt.title("Price Variation by Travel Class")
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='stops', y='price', data=train_df, palette='Set3')
plt.title("Flight Price vs Number of Stops")
plt.show()

In [ ]:
# Group by days_left and calculate average price
days_price = train_df.groupby('days_left')['price'].mean().reset_index()

# Plot line chart
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
sns.lineplot(x='days_left', y='price', data=days_price, marker='o', color='blue')
plt.title("Days Left vs Average Ticket Price")
plt.xlabel("Days Left to Departure")
plt.ylabel("Average Price (₹)")
plt.grid(True)
plt.tight_layout()
plt.show()

# Scale Numerical features and Encode Categorical features

In [ ]:
num_cols = X.select_dtypes(include=['int64', 'float64']).drop(columns='id', errors='ignore').columns
cat_cols = X.select_dtypes(include=['object']).columns

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], verbose_feature_names_out=False)

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score

X = train_df_t.drop(['id'], axis=1)
y = y_train

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### 1. Linear Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

pipeline_lr = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

pipeline_lr.fit(X, y)
pred_lr = pipeline_lr.predict(X_val)
score_lr = r2_score(y_val, pred_lr)
print(f"Linear Regression: R2 Score = {score_lr:.4f}")

### 2.  Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

pipeline_dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])

pipeline_dt.fit(X, y)
pred_dt = pipeline_dt.predict(X_val)
score_dt = r2_score(y_val, pred_dt)
print(f"Decision Tree: R2 Score = {score_dt:.4f}")

### 3. K Neighbors Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

pipeline_knn = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', KNeighborsRegressor())
])

pipeline_knn.fit(X, y)
pred_knn = pipeline_knn.predict(X_val)
score_knn = r2_score(y_val, pred_knn)
print(f"KNN Regressor: R2 Score = {score_knn:.4f}")

### 4. XGB Regressor

In [ ]:
from xgboost import XGBRegressor

pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor())
])

pipeline_xgb.fit(X, y)
pred_xgb = pipeline_xgb.predict(X_val)
score_xgb = r2_score(y_val, pred_xgb)
print(f"XGBoost: R2 Score = {score_xgb:.4f}")

## With Hyperparameter Tuning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score

# 30% sample for tuning
X_sample = X.sample(frac=0.3, random_state=42)
y_sample = y.loc[X_sample.index]

### 5. Decision Tree Regressor with tuning

In [ ]:
from sklearn.tree import DecisionTreeRegressor

pipe_dt = Pipeline([
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])

params_dt = {
    'model__max_depth': [3, 5, 10],
    'model__min_samples_split': [2, 5]
}

grid_dt = RandomizedSearchCV(pipe_dt, params_dt, n_iter=5, cv=3, scoring='r2', n_jobs=-1)
grid_dt.fit(X_sample, y_sample)

print("Decision Tree Best:", grid_dt.best_params_)
print("Score (30% CV):", grid_dt.best_score_)

best_dt = grid_dt.best_estimator_
best_dt.fit(X, y)

### 6. Random Forest Regressor with tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor

pipe_rf = Pipeline([
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_jobs=-1, random_state=42))
])

params_rf = {
    'model__n_estimators': [10, 30],
    'model__max_depth': [5, 10],
    'model__min_samples_split': [5, 10]
}

grid_rf = RandomizedSearchCV(pipe_rf, params_rf, n_iter=5, cv=3, scoring='r2', n_jobs=-1)
grid_rf.fit(X_sample, y_sample)

print("Random Forest Best:", grid_rf.best_params_)
print("Score (30% CV):", grid_rf.best_score_)

best_rf = grid_rf.best_estimator_
best_rf.fit(X, y)

### 7. Gradient Boosting Regressor with tuning

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

pipe_gb = Pipeline([
    ('preprocessor', preprocessor),
    ('model', GradientBoostingRegressor())
])

params_gb = {
    'model__n_estimators': [50, 100],
    'model__learning_rate': [0.05, 0.1],
    'model__max_depth': [3, 5]
}

grid_gb = RandomizedSearchCV(pipe_gb, params_gb, n_iter=5, cv=3, scoring='r2', n_jobs=-1)
grid_gb.fit(X_sample, y_sample)

print("Gradient Boosting Best:", grid_gb.best_params_)
print("Score (30% CV):", grid_gb.best_score_)

best_gb = grid_gb.best_estimator_
best_gb.fit(X, y)

### 8. Lasso Regression with tuning

In [ ]:
from sklearn.linear_model import Lasso

pipe_lasso = Pipeline([
    ('preprocessor', preprocessor),
    ('model', Lasso())
])

params_lasso = {
    'model__alpha': [0.001, 0.01, 0.1, 1.0]
}

grid_lasso = RandomizedSearchCV(pipe_lasso, params_lasso, n_iter=5, cv=3, scoring='r2', n_jobs=-1)
grid_lasso.fit(X_sample, y_sample)

print("Lasso Best:", grid_lasso.best_params_)
print("Score (30% CV):", grid_lasso.best_score_)

best_lasso = grid_lasso.best_estimator_
best_lasso.fit(X, y)

### 9. SVR with tuning

In [ ]:
# from sklearn.svm import SVR

# pipe_svr = Pipeline([
#     ('preprocessor', preprocessor),
#     ('model', SVR())
# ])

# params_svr = {
#     'model__C': [1, 10],
#     'model__kernel': ['linear', 'rbf']
# }

# grid_svr = RandomizedSearchCV(pipe_svr, params_svr, n_iter=5, cv=3, scoring='r2', n_jobs=-1)
# grid_svr.fit(X_sample, y_sample)

# print("SVR Best:", grid_svr.best_params_)
# print("Score (30% CV):", grid_svr.best_score_)

# best_svr = grid_svr.best_estimator_
# best_svr.fit(X, y)

# Submiting Best Model

In [ ]:
from sklearn.tree import DecisionTreeRegressor

pipeline_dt = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor())
])

pipeline_dt.fit(X, y)
pred_dt = pipeline_dt.predict(X_val)
score_dt = r2_score(y_val, pred_dt)
print(f"Decision Tree: R2 Score = {score_dt:.4f}")

In [ ]:
import pandas as pd

# Predict (pipeline already includes preprocessor)
y_pred_log = pipeline_dt.predict(test_df_t)

# Inverse transform to get back original price
y_pred_price = np.expm1(y_pred_log)

# Create submission
submission = pd.DataFrame({
    'id': test_df['id'],
    'price': y_pred_price
})

submission.to_csv('submission.csv', index=False)
print("✅ Submission file created: submission.csv")
